In [1]:
import os

In [2]:
%pwd

'd:\\project\\Predict-Lung-Disease\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\project\\Predict-Lung-Disease'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=config.root_dir,
            tensorboard_root_log_dir=config.tensorboard_root_log_dir,
            checkpoint_model_filepath=config.checkpoint_model_filepath
        )

        return prepare_callback_config

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_split.root_dir)
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=training.root_dir,
            trained_model_path=training.trained_model_path,
            updated_base_model_path=prepare_base_model.updated_base_model_path,
            training_data=training_data,
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE
        )

        return training_config

In [8]:
import torch
import time
from torch.utils.tensorboard import SummaryWriter

In [9]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir, 
            f"tb_logs_at_{timestamp}"
        )

        return SummaryWriter(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        def save_checkpoint(model, optimizer, epoch, loss):
            checkpoint = {
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'epoch': epoch,
                'loss': loss
            }
            torch.save(checkpoint, self.config.checkpoint_model_filepath)
        return save_checkpoint

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import os
from pathlib import Path
from torch.utils.data import random_split

In [11]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    def get_base_model(self):
        # Tải mô hình với strict=False để tránh lỗi thiếu key
        self.model = models.vgg16()
        self.model.load_state_dict(torch.load(self.config.updated_base_model_path), strict=False)
        self.model.to(self.device)

    def train_valid_loader(self):
        data_transforms = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(self.config.params_image_size[0]),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(40),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]) if self.config.params_is_augmentation else transforms.Compose([
                transforms.Resize(self.config.params_image_size[0]),
                transforms.CenterCrop(self.config.params_image_size[0]),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            'val': transforms.Compose([
                transforms.Resize(self.config.params_image_size[0]),
                transforms.CenterCrop(self.config.params_image_size[0]),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        }

        # Tải dữ liệu từ các thư mục đã chia sẵn
        train_dataset = datasets.ImageFolder(self.config.training_data + "/train", transform=data_transforms['train'])
        val_dataset = datasets.ImageFolder(self.config.training_data + "/val", transform=data_transforms['val'])

        # Tạo DataLoader cho các tập train và val
        self.train_loader = DataLoader(train_dataset, batch_size=self.config.params_batch_size, shuffle=True)
        self.val_loader = DataLoader(val_dataset, batch_size=self.config.params_batch_size, shuffle=False)

    @staticmethod
    def save_model(path: Path, model: nn.Module):
        torch.save(model.state_dict(), path)

    def train(self, callback_list=None):
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(self.model.parameters(), lr=self.config.params_learning_rate, momentum=0.9)
        
        # Training loop
        self.model.train()
        for epoch in range(self.config.params_epochs):
            running_loss = 0.0
            for inputs, labels in self.train_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                # Zero gradients
                optimizer.zero_grad()

                # Forward + backward + optimize
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item() * inputs.size(0)
            epoch_loss = running_loss / len(self.train_loader.dataset)
            print(f'Epoch {epoch + 1}/{self.config.params_epochs}, Loss: {epoch_loss:.4f}')

            # Validation
            self.model.eval()
            val_loss = 0.0
            correct = 0
            total = 0
            with torch.no_grad():
                for inputs, labels in self.val_loader:
                    inputs, labels = inputs.to(self.device), labels.to(self.device)
                    outputs = self.model(inputs)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item() * inputs.size(0)
                    _, predicted = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            val_loss /= len(self.val_loader.dataset)
            accuracy = 100 * correct / total
            print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')

            # Trở lại chế độ train
            self.model.train()

        # Lưu mô hình đã huấn luyện
        self.save_model(path=self.config.trained_model_path, model=self.model)

In [ ]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_loader()
    training.train(
        callback_list=callback_list
    )
except Exception as e:
    raise e

[2024-11-17 11:21:59,367: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-17 11:21:59,372: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-17 11:21:59,373: INFO: common: creating directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-11-17 11:21:59,375: INFO: common: creating directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2024-11-17 11:21:59,440: INFO: common: creating directory at: artifacts\training]


C:\Users\Account_Test\AppData\Local\Temp\ipykernel_16276\608002546.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(self.config.upda